# LAB 3: Regresja wieloraka

In [253]:
#Aleksandra Kowalczyk
# 307414

Prognozowanie cen domów (wiele zmiennych)

W tym notatniku wykorzystamy dane dotyczące sprzedaży domów. Przewidywanie ich ceny zostanie dokonane przy pomocy regresji wielorakiej. Pierwsze zadanie dotyczy eksploracji regresji wielorakiej poprzez opracowanie nowych cech i pomiar błędu. W drugim zadaniu zaimplementować należy algorytm spadku gradientu.

     Użyj wbudowanych funkcji tworzenia wykresów (lub w inny sposób), aby obliczyć wagi regresji (współczynniki)
     Biorąc pod uwagę wagi regresji, predyktory i wynik napisz funkcję obliczającą resztkową sumę kwadratów
     Spójrz na współczynniki i zinterpretuj ich znaczenie
     Oceń wiele modeli za pomocą RSS

# Potrzebne biblioteki

In [2]:
import pandas as pd
import numpy as np

# Wczytanie danych ceny domów 

Zestaw danych pochodzi ze sprzedaży domów w King County, regionie, w którym znajduje się miasto Seattle w stanie Waszyngton.

In [254]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
domy = pd.read_csv('kc_house_data.csv',sep=',', dtype=dtype_dict)

In [255]:
domy.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Podziel dane na uczące i testowe.
Używamy seed = 0, aby każdy, kto korzysta z tego notebooka, uzyskał te same wyniki. W praktyce możesz ustawić losowy podział.

In [256]:
y = domy['price']
X = domy.drop(['price'], axis=1)
features = X.columns.values
features 

array(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], dtype=object)

In [257]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Nauka modelu regresji wielorakiej

Wykorzystując cechy 'sqft_living', 'bedrooms', 'bathrooms' uczymy nasz model.

In [258]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']

data = X_train[example_features]
list_data = list(data)

my_X = pd.DataFrame({'X_0' :np.ones((len(data))), 'sqft_living':data[list_data[0]],'bedrooms':data[list_data[1]],'bathrooms':data[list_data[2]] })
i = 0


pinw = np.linalg.pinv(my_X)
w = np.dot(pinw,y_train)

Po dopasowaniu modelu możemy wyodrębnić współczynniki (wagi) modelu regresji:

In [259]:
print('Otrzymane współczynniki:', w)

Otrzymane współczynniki: [ 67512.01513809    313.17055038 -56754.66651422   6887.71910816]


# Przewidywanie wartości modelu

Mając wyliczone parametry modelu napisz funkcję do przewidywania wartości dla zadanego modelu

In [260]:
def predict_output(feature_matrix, weights):
    
    predictions_output = np.dot(feature_matrix, weights)
    return(predictions_output)

In [261]:
data = X_train[example_features]
my_X = pd.DataFrame({'sqft_living':data[list_data[0]],'bedrooms':data[list_data[1]],'bathrooms':data[list_data[2]] })
my_X.insert(loc= 0,column= 'X_0', value = np.ones((len(my_X))))
predict_output(my_X,w)

array([395813.49880289, 471910.89304528, 248935.49417374, ...,
       651827.88248964, 596482.9917022 , 601336.62698305])

# Wyliczamy błąd (SSE -  sum of squared estimate of errors)

Teraz, gdy możemy wykonać przewidywania na podstawie modelu, napiszmy funkcję obliczającą RSS modelu. Wykonaj poniższą funkcję, aby obliczy sumę kwadratów błędu estymacji (SSE) na podstawie modelu, danych i wyniku.

In [262]:
def policz_SSE(w, data, outcome):
    
    # Wylicz przewidywaną wartość
    data.insert(loc= 0,column= 'X_0', value = np.ones((len(data))))

    f = predict_output(data,w)

    # Policz błąd bezwzglęny
    e_abs = f-outcome
    
    # Podnieś błędy do kwadratu i zsumuj je
    SSE = np.sum(e_abs**2)

    return(SSE)    

Przetestuj swoją funkcję obliczając błąd SSE z danych TEST dla przykładowego modelu:

In [263]:
# testuję dla danych TEST:
data = X_test[example_features]
# współczynniki mam już wyliczone powyżej

rss_example_test = policz_SSE(w, data, y_test)
print(rss_example_test) 

259213572106088.06


# Utwórz nowe cechy

Mimo iż nasz model regresji wielorakiej obejmuje wiele różnych cech (np. ilosc_sypiani, powierzchnia i ilosc_lazienek) możemy również rozważyć przekształcenie istniejących cech, np. log(powierzchnia), czy nawet mnożenie ilości sypialni i łazienek.

Użyjemy funkcji logarytmu, aby utworzyć nowe cechy, więc najpierw powinieneś importujemy ją z biblioteki matematycznej.

In [18]:
from math import log

Następnie utwórz następujące 4 nowe cechy jako kolumny w danych TRENINGOWYCH i TESTOWYCH:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
Jako przykład oto pierwsza:

In [264]:
X_train['bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x**2)
X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)

X_test

<ipython-input-264-007d62de25dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x**2)
<ipython-input-264-007d62de25dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared
17384,1453602313,20141029T000000,2.0,1.50,1430.0,1650,3,0,0,3,...,1430,0,1999,0,98125,47.7222,-122.290,1430.0,1650.0,4.0
722,2225059214,20140808T000000,4.0,3.25,4670.0,51836,2,0,0,4,...,4670,0,1988,0,98005,47.6350,-122.164,4230.0,41075.0,16.0
2680,2768000270,20140625T000000,2.0,0.75,1440.0,3700,1,0,0,3,...,1200,240,1914,0,98107,47.6707,-122.364,1440.0,4300.0,4.0
18754,6819100040,20140624T000000,2.0,1.00,1130.0,2640,1,0,0,4,...,1130,0,1927,0,98109,47.6438,-122.357,1680.0,3200.0,4.0
14554,4027700666,20150426T000000,4.0,2.50,3180.0,9603,2,0,2,3,...,3180,0,2002,0,98155,47.7717,-122.277,2440.0,15261.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,3528000545,20140815T000000,4.0,3.25,3090.0,67518,2,0,0,3,...,3090,0,1988,0,98053,47.6674,-122.046,3200.0,65775.0,16.0
16547,0526059259,20140819T000000,3.0,1.75,1260.0,8487,1,0,0,3,...,1260,0,1970,0,98011,47.7664,-122.201,1890.0,13051.0,9.0
4585,0339600090,20140925T000000,3.0,2.50,1360.0,3718,2,0,0,3,...,1360,0,1987,0,98052,47.6827,-122.097,1090.0,3718.0,9.0
17762,7750500120,20141118T000000,3.0,1.00,950.0,4760,1.5,0,0,3,...,950,0,1929,0,98106,47.5236,-122.348,1080.0,4760.0,9.0


In [265]:
# utwórz pozostałe 3 funkcje zarówno w danych TRENINGOWYCH, jak i TESTOWYCH

#druga funkcja
X_train['bed_bath_rooms'] = X_train['bedrooms'] * X_train['bathrooms']
X_test['bed_bath_rooms'] = X_test['bedrooms'] * X_test['bathrooms']

#trzecia funkcja
X_train['log_sqft_living'] = X_train['sqft_living'].apply(lambda x: log(x))
X_test['log_sqft_living'] = X_test['sqft_living'].apply(lambda x: log(x))

#czwarta funkcja
X_train['lat_plus_long'] = X_train['lat'] + X_train['long']
X_test['lat_plus_long'] =  X_test['lat'] + X_test['long']

X_test

<ipython-input-265-a295e031e5c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['bed_bath_rooms'] = X_train['bedrooms'] * X_train['bathrooms']
<ipython-input-265-a295e031e5c4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['bed_bath_rooms'] = X_test['bedrooms'] * X_test['bathrooms']
<ipython-input-265-a295e031e5c4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
17384,1453602313,20141029T000000,2.0,1.50,1430.0,1650,3,0,0,3,...,0,98125,47.7222,-122.290,1430.0,1650.0,4.0,3.00,7.265430,-74.5678
722,2225059214,20140808T000000,4.0,3.25,4670.0,51836,2,0,0,4,...,0,98005,47.6350,-122.164,4230.0,41075.0,16.0,13.00,8.448914,-74.5290
2680,2768000270,20140625T000000,2.0,0.75,1440.0,3700,1,0,0,3,...,0,98107,47.6707,-122.364,1440.0,4300.0,4.0,1.50,7.272398,-74.6933
18754,6819100040,20140624T000000,2.0,1.00,1130.0,2640,1,0,0,4,...,0,98109,47.6438,-122.357,1680.0,3200.0,4.0,2.00,7.029973,-74.7132
14554,4027700666,20150426T000000,4.0,2.50,3180.0,9603,2,0,2,3,...,0,98155,47.7717,-122.277,2440.0,15261.0,16.0,10.00,8.064636,-74.5053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,3528000545,20140815T000000,4.0,3.25,3090.0,67518,2,0,0,3,...,0,98053,47.6674,-122.046,3200.0,65775.0,16.0,13.00,8.035926,-74.3786
16547,0526059259,20140819T000000,3.0,1.75,1260.0,8487,1,0,0,3,...,0,98011,47.7664,-122.201,1890.0,13051.0,9.0,5.25,7.138867,-74.4346
4585,0339600090,20140925T000000,3.0,2.50,1360.0,3718,2,0,0,3,...,0,98052,47.6827,-122.097,1090.0,3718.0,9.0,7.50,7.215240,-74.4143
17762,7750500120,20141118T000000,3.0,1.00,950.0,4760,1.5,0,0,3,...,0,98106,47.5236,-122.348,1080.0,4760.0,9.0,3.00,6.856462,-74.8244


* Kwadrat sypialni zwiększa separację między nielicznymi sypialniami (np. 1) i wieloma sypialniami (np. 4), ponieważ 1 ^ 2 = 1, ale 4 ^ 2 = 16. W konsekwencji ta funkcja będzie miała wpływ głównie na domy z wieloma sypialniami.
* Sypialnia razy łazienka daje tak zwaną funkcję „interakcji”. Wynik jest wysoki, gdy  wartościu *obu* cech są duże.
* Przejęcie logarytmu stóp kwadratowych powoduje zbliżenie dużych wartości i rozłożenie małych wartości. Wynika to z reguły prawoskośności posiadanych danych/posiadanego atrybutu.
* Dodawanie szerokości do długości geograficznej jest całkowicie bezsensowne, ale i tak to zrobimy (zobaczymy później dlaczego)

**Pytanie quizu: Jaka jest średnia (średnia arytmetyczna) twoich 4 nowych funkcji w danych TEST? (w zaokrągleniu do 2 cyfr)**

In [266]:
arytmetyczna_1 = np.sum(X_test['bedrooms_squared'])/ len(X_test['bedrooms_squared'])
print("średnia arytmetyczna 1 funkcji")
print(round(arytmetyczna_1,2))

arytmetyczna_2 = np.sum(X_test['bed_bath_rooms'])/ len(X_test['bed_bath_rooms'])
print("średnia arytmetyczna 2 funkcji")
print(round(arytmetyczna_2,2))

arytmetyczna_3 = np.sum(X_test['log_sqft_living'])/ len(X_test['log_sqft_living'])
print("średnia arytmetyczna 3 funkcji")
print(round(arytmetyczna_3,2))

arytmetyczna_4 = np.sum(X_test['lat_plus_long'])/ len(X_test['lat_plus_long'])
print("średnia arytmetyczna 4 funkcji")
print(round(arytmetyczna_4,2))

# sprawdź jeszze
print("średnia arytmetyczna wszystkich czterech funkcji")
arytmetyczna_all = (arytmetyczna_1 + arytmetyczna_2 + arytmetyczna_3 + arytmetyczna_4) / 4
print(round(arytmetyczna_all,2))

średnia arytmetyczna 1 funkcji
12.21
średnia arytmetyczna 2 funkcji
7.45
średnia arytmetyczna 3 funkcji
7.55
średnia arytmetyczna 4 funkcji
-74.65
średnia arytmetyczna wszystkich czterech funkcji
-11.86


# Uczenie wielu modeli

Teraz poznamy wagi trzech (zagnieżdżonych) modeli do przewidywania cen domów. Pierwszy model będzie miał najmniej cech, drugi model doda jedną cechę, a trzeci doda jeszcze kilka:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: + bedrooms\*bathrooms
* Model 3: + log squarefeet, bedrooms squared, i (bezsensowne) latitude + longitude

In [267]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Teraz, gdy mamy już cechy, poznaj wagi trzech różnych modeli do przewidywania docelowej = „ceny” za pomocą funkcji model_train i spójrz na wartość wag/współczynników:

In [268]:
# Funkcja szukająca współczynników modelu

def model_train(X_train, y_train):
    
    # Wylicz współczynniki modelu wykorzystując pseudoodwrotność

    my_X = X_train
    my_X.insert(loc= 0,column= 'X_0', value = np.ones((len(my_X))))

    model = np.dot(np.linalg.pinv(my_X),y_train)
    
    return(model)

In [269]:
# Wyucz 3 modele:
model_1 =model_train(X_train[model_1_features], y_train)
model_2 =model_train(X_train[model_2_features], y_train)
model_3 =model_train(X_train[model_3_features], y_train)



In [270]:
# Wyświetl wagi współczynników modelu:

print(model_1)
print(model_2)
print(model_3)

[-7.08708462e+07  3.12942010e+02 -5.30962691e+04  1.47770428e+04
  6.53983343e+05 -3.25707336e+05]
[-6.86068204e+07  3.06819573e+02 -1.04604718e+05 -7.01815289e+04
  6.50590952e+05 -3.09965751e+05  2.49441497e+04]
[-6.26284503e+07  5.37808086e+02  2.78047910e+03  1.01363766e+05
  5.30798406e+05 -4.09655436e+05 -1.81822552e+04  7.24579939e+02
 -5.71030023e+05  1.21142971e+05]


**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 1?**

**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 2?**

Zastanów się, co to znaczy.

In [271]:
#dodatni dla pierwszego modelu
#ujemny dla drugiego modelu

# znak zmienia się z dodatniego na ujemny, gdy dodajemy cechę 'bed_bath_rooms', która przyjmuje dodatni znak;
# oznacza to że waga samych 'łazieniek' spada (mniej istotna jest teraz)
# gdy bierzemy pod uwage jeszcze 'lazienki_symialnie' która ma dużą wagę i jest istotniejszą cechą

# Porównywanie wielu modeli

Teraz, gdy otrzymaliśmy trzy modele i wyodrębniliśmy wagi modeli, chcemy ocenić, który model jest najlepszy.

Używając wcześniej naopisanych funkcji obliczy SSE dla danych uczących dla każdego z trzech modeli.

**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TRENINGOWYCH?** Czy tego się spodziewałeś/-łaś?

In [272]:
# Policz SSE na danych TRENINGOWYCH dla każdeg z 3 modeli i zapisz wartości:
train_1_SSE = policz_SSE(model_1,X_train[model_1_features], y_train )
print("błąd dla modelu 1 dane treningowe")
print(train_1_SSE)

train_2_SSE = policz_SSE(model_2,X_train[model_2_features], y_train )
print("błąd dla modelu 2 dane treningowe")
print(train_2_SSE)

train_3_SSE = policz_SSE(model_3,X_train[model_3_features], y_train )
print("błąd dla modelu 3 dane treningowe")
print(train_3_SSE)

błąd dla modelu 1 dane treningowe
979843597588332.1
błąd dla modelu 2 dane treningowe
970799199729580.6
błąd dla modelu 3 dane treningowe
913653644974959.1


In [273]:
# najmniejszy błąd SSE jest dla modelu 3 (widzimy że dla modelu 1 jest największy, potem spada dla 2 modelu i dla 3 najmniejszy)

# jest to wynik, którego się spodziewałam, ponieważ do modelu trzeciego dodałam więcej cech, 
# dzięki czemu model mógł lepiej zostać wyuczony, dodatkowo dodaliśmy te przekształcone cechy,
# czyli te, które dzięki przekształceniom są bardziej wyraźne, pozwaljają przygotować bardziej dobrany model, mówią nam więcej o mieszkaniu (w większości przypadków, nie dotyczy to np lat_plus_long)


Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli.

In [274]:
# Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli i zapisz wartości:
test_1_SSE = policz_SSE(model_1,X_test[model_1_features], y_test )
print("błąd dla modelu 1 dane testowe")
print(test_1_SSE)

test_2_SSE = policz_SSE(model_2,X_test[model_2_features], y_test )
print("błąd dla modelu 2 dane testowe")
print(test_2_SSE)

test_3_SSE = policz_SSE(model_3,X_test[model_3_features], y_test )
print("błąd dla modelu 3 dane testowe")
print(test_3_SSE)

błąd dla modelu 1 dane testowe
213487129319106.7
błąd dla modelu 2 dane testowe
210778544168944.12
błąd dla modelu 3 dane testowe
203972051917619.94


**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TESTOWYCH?** Czy tego się spodziewałeś/-łaś? Pomyśl o cechach, które zostały dodane do każdego z modeli.

In [275]:
# najmniejszy błąd SSE jest dla modelu 3 (widzimy że dla modelu 1 jest największy, potem spada dla 2 modelu i dla 3 najmniejszy)
# mamy do czynienia z tą samą sytuacją jak powyżej, lepiej wyuczony model pozwala nam osiągnać dla danych testowych mniejszy błąd

# Policz pochodną

Przejdziemy teraz do obliczania pochodnej funkcji kosztu regresji. Przypomnij sobie, że funkcja kosztu jest sumą kwadratów różnic między punktami danych a przewidywanym wynikiem.

Ponieważ pochodna sumy jest sumą pochodnych, możemy obliczyć pochodną dla pojedynczego punktu danych, a następnie zsumować na podstawie punktów danych. Możemy zapisać kwadratową różnicę między obserwowanym a przewidywanym wynikiem dla pojedynczego punktu w następujący sposób:

(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)^2

Gdzie mamy *k* cech i stałą. Tak więc pochodną w odniesieniu do wagi w[i] według reguły łańcucha jest:

2\*(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)\* [cecha_i]

Pojęcie w nawiasach to tylko błąd (różnica między prognozowaniem a wyjściem). Możemy więc ponownie napisać to jako:

2\*error\*[cecha_i]

Oznacza to, że pochodną wagi cechy *i* jest suma (ponad punkty danych) 2-krotności iloczynu błędu i samej cechy. W przypadku stałej jest to tylko dwukrotność sumy błędów!

Przypomnijmy, że dwukrotność sumy iloczynu dwóch wektorów jest tylko dwukrotnością iloczynu dwóch wektorów. Dlatego pochodna wagi dla *cechy_i* jest tylko dwukrotnością iloczynu między wartościami *cechy_i* a bieżącymi błędami.

Mając to na uwadze, należy wykonać następującą funkcję pochodną, która oblicza pochodną współczynnika na podstawie wartości cechy (we wszystkich punktach danych) i błędów (we wszystkich punktach danych).

In [276]:
def feature_derivative(errors, feature):
    
    # Oblicz podwojony iloczyn cech i błędów, a następnie zwróć otrzymną wartość
    derivative = 2*np.dot(errors, feature)
    
    return(derivative)

# Metoda spadku gradientu / gradientu prostego (Gradient Descent)

Teraz napiszemy funkcję, która wykonuje spadek gradientu. Podstawową przesłanką jest prosta. Biorąc pod uwagę punkt początkowy, aktualizujemy bieżące wagi, przesuwając się w kierunku ujemnego gradientu. Przypomnijmy, że gradient jest kierunkiem *wzrostu*, a zatem gradient ujemny jest kierunkiem *spadku* i staramy się *zminimalizować* funkcję kosztu.

Współczynnik, z jakim poruszamy się w kierunku gradientu ujemnego, nazywa się „rozmiarem kroku” - $\alpha$. Zatrzymujemy się, gdy jesteśmy „wystarczająco blisko” do rozwiązania optymalnego. Definiujemy to, wymagając, aby wielkość (długość) wektora gradientu była mniejsza niż stała „tolerancja” - $\epsilon$.

Mając to na uwadze, wykonaj poniższą funkcję spadku gradientu poniżej, używając powyższej funkcji pochodnej. Dla każdego kroku zejścia gradientu aktualizujemy wagę każdej funkcji przed obliczeniem naszych kryteriów zatrzymania

In [277]:
from math import sqrt # wywołanie funkcji na wektorze [g[0], g[1], g[2]] daje sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [278]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # miej pewność, że wagi są typu numpy array
    while not converged:
        # obliczyć prognozy na podstawie feature_matrix i wag za pomocą funkcji predict_output()
        predictions = predict_output(feature_matrix, weights)

        # wylicz błąd predykcji jako: predict_output - output
        errors = predictions - output

        gradient_sum_squares = 0 # inicjuj sumę pierwiastków gradientów
        # chociaż nie osiągnęliśmy jeszcze tolerancji, zaktualizuj wagę każdej funkcji
        for i in range(len(weights)): # zapętlenie każdej wagi
            # Przypomnij sobie, że feature_matrix [:, i] to kolumna cech powiązana z wagami[i]
            # obliczyć pochodną dla wagi[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            # dodaj kwadrat wartości pochodnej do sumy kwadratów gradientów (ocena zbieżności) 
            gradient_sum_squares += (derivative**2)
            # odejmij wielkość kroku pomnożoną przez pochodną od bieżącej wagi
            weights[i] -= (step_size * derivative)         
            
        # oblicz pierwiastek kwadratowy z sumy gradientów kwadratów, aby uzyskać wielkość gradientu:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

Kilka rzeczy, na które należy zwrócić uwagę, zanim zaczniemy korzystać z metody spadku gradientu. Ponieważ gradient jest sumą wszystkich punktów danych i obejmuje iloczyn błędu i wartości cechy, sam gradient będzie bardzo duży, ponieważ cechy są duże (stopy kwadratowe), a wynik jest duży (ceny). Tak więc, chociaż można oczekiwać, że „tolerancja” będzie niewielka, mała jest jedynie zależna od wielkości cechy.

Z podobnych powodów rozmiar kroku będzie znacznie mniejszy niż można się spodziewać, ale dzieje się tak, ponieważ gradient ma tak duże wartości.

# Uruchamianie spadku gradientu jako prostej regresji

Chociaż opadanie gradientu jest zaprojektowane dla regresji wielorakiej, ponieważ stała jest teraz funkcją, możemy użyć funkcji spadku gradientu do oszacowania parametrów prostej regresji na "squarefeet". Następująca komórka ustawia funkcję parametr_macierz, wynik, wagi początkowe i rozmiar kroku dla pierwszego modelu:

In [279]:
# usunęłam z get_numpy_data output - ponieważ nie wyszukiwało nazwy 'price', a to dlatego że na samym początku
# zadania usunęliśmy tę kolumnę z X, więc output w tej funkcji nie działał poprawnie

def get_numpy_data(train_data, simple_features):
    simple_feature_matrix = train_data[simple_features] 
    simple_feature_matrix['X_0'] = 1
    simple_features.insert(0, 'X_0')
    simple_feature_matrix = simple_feature_matrix[simple_features]
    simple_feature_matrix = np.array(simple_feature_matrix)

    return(simple_feature_matrix)

In [298]:
# test
simple_features = ['sqft_living']

my_output = y_train
(simple_feature_matrix) = get_numpy_data(X_train, simple_features)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

simple_feature_matrix

<ipython-input-279-37c5326bc417>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_feature_matrix['X_0'] = 1


array([[1.00e+00, 1.57e+03],
       [1.00e+00, 1.78e+03],
       [1.00e+00, 1.09e+03],
       ...,
       [1.00e+00, 2.36e+03],
       [1.00e+00, 2.37e+03],
       [1.00e+00, 2.38e+03]])

Następnie uruchom gradient z powyższymi parametrami.

In [299]:
test_weight = regression_gradient_descent(simple_feature_matrix, my_output, initial_weights, step_size, tolerance)
print("Otrzymane współczynniki dla metody spadku gradientu")
test_weight

Otrzymane współczynniki dla metody spadku gradientu


array([-46999.88720259,    283.46383063])

Porównaj wagi do tych uzyskanych przy pomocy pseudoodwrotności?

**Pytanie quizu: Jaka jest waga sqft_living - drugi element „simple_weights” (w zaokrągleniu do 1 miejsca po przecinku)?**

In [300]:
simple_features = ['sqft_living']
my_x_data = X_train[simple_features]

ps =model_train(my_x_data, y_train)
print("Otrzymane współczynniki przy pomocy pseudoodwrotności")
ps

Otrzymane współczynniki przy pomocy pseudoodwrotności


array([-48257.06359103,    283.96855716])

In [234]:
# odpowiedź quiz
#dla metody spadku gradientu waga dla sqft_living wynosi około 283.5, zaś wag uzyskana pseudoodwrotności około 284
# są więc one bardzo do siebie zbliżone, waga w0 dla obydwóch metod również jest bardzo podobna

Zobacz jak się zachowuje metoda gradientowa po przeskalowaniu wartości cech:

In [301]:
# dokonam przeskalowania skalarem MinMax

In [302]:
from sklearn.preprocessing import MinMaxScaler
standscaler = MinMaxScaler()

# funkcja do przeskalowania wartosci X
def get_numpy_data_scaler(train_data, simple_features):
    
    simple_feature_matrix = train_data[simple_features] 
    simple_feature_matrix[simple_features] = standscaler.fit_transform(simple_feature_matrix[simple_features])
    simple_feature_matrix['X_0'] = 1
    simple_features.insert(0, 'X_0')
    simple_feature_matrix = simple_feature_matrix[simple_features]
    simple_feature_matrix = np.array(simple_feature_matrix)

    return(simple_feature_matrix)


In [303]:
simple_features = ['sqft_living']
my_output = y_train
# przeskalowanie wartości X poprzez funkcje get_numpy_data_scaler:
(simple_feature_matrix) = get_numpy_data_scaler(X_train, simple_features)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

simple_feature_matrix

<ipython-input-302-e85a6e44bcb6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_feature_matrix[simple_features] = standscaler.fit_transform(simple_feature_matrix[simple_features])
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

array([[1.        , 0.09660377],
       [1.        , 0.11245283],
       [1.        , 0.06037736],
       ...,
       [1.        , 0.15622642],
       [1.        , 0.15698113],
       [1.        , 0.15773585]])

In [304]:
#przeskalowanie wartości Y:
my_output = np.array(my_output)
my_output = my_output.reshape(-1,1)
my_output = standscaler.fit_transform(my_output)
my_output

array([[0.05508197],
       [0.07344262],
       [0.04032787],
       ...,
       [0.04668852],
       [0.04406557],
       [0.0819541 ]])

In [ ]:
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

test_weight = regression_gradient_descent(simple_feature_matrix, my_output, initial_weights, step_size, tolerance)
test_weight

In [251]:
# powyższe polecenie test_weight uruchamia się, jednak zapętla się w programie i nie kończy 
#nie uzyskujemy odpowiedzi 
# oznacza to, że dla podanych danych gradient nigdy nie jest poniżej wartości tolerancji więc program 
# nie może się zakończyć

In [252]:
# by móc uzyskać wynik powinniśmy zmienić wartości step_size i tolerance
# jeśli prawidłowo dobralibyśmy wartość step_size to wartość tolerance powinna wyjść nam mała
# ze względu na to, że sam gradient będzie o wiele mniejszy teraz po przeskalowaniu niż dla poprzednich wartości,
# ponieważ cechy są o wiele mniejsze (stopy kwadratowe), a także wynik jest o wiele mniejszy (ceny). 
# co potwierdza fakt, że „tolerancja” jest zależna od wielkości cechy.

In [ ]:
#koniec